In [40]:
import pandas as pd
import re

from tqdm.auto import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from actions import remove_typos
import actions
import importlib

importlib.reload(actions)

<module 'actions' from 'd:\\GitHub\\AeroMetrics\\pipeline\\actions\\__init__.py'>

In [54]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")

parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")
test = False
if test:
    parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv_test.parquet")
    parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv_test.parquet")
    parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv_test.parquet")
print(
    parquet_file_abcd,
    parquet_file_abcd_conv,
    parquet_file_mnop,
    parquet_file_mnop_conv,
    parquet_file_zyxw,
    parquet_file_zyxw_conv,
)

..\data\ABCD_tripfiles.parquet ..\data\ABCD_tripfiles_conv.parquet ..\data\MNOP_tripfiles.parquet ..\data\MNOP_tripfiles_conv.parquet ..\data\ZYXW_tripfiles.parquet ..\data\ZYXW_tripfiles_conv.parquet


In [55]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

In [4]:
# print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
# print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
# print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

In [56]:
action_extractors = {
    "CalculateWeightAndTrimAction": actions.CalculateWeightAndTrimAction.extract,
    "CheckinMsgProcessor": actions.CheckinMsgProcessor.extract,
    "CreateLoadingInstructionAction": actions.CreateLoadingInstructionAction.extract,
    "CreateLoadsheetAction": actions.CreateLoadsheetAction.extract,
    "CreateZFWMessageAction": actions.CreateZFWMessageAction.extract,
    "EstimateStorePaxDataAction": actions.EstimateStorePaxDataAction.extract,
    "RampFinalAction": actions.RampFinalAction.extract,
    "SendFuelOrderAction": actions.SendFuelOrderAction.extract,
    "SendLoadingInstructionAction": actions.SendLoadingInstructionAction.extract,
    "SendLoadsheetAction": actions.SendLoadsheetAction.extract,
    "SetActualBagWeightIndicatorAction": actions.SetActualBagWeightIndicatorAction.extract,
    "SetCKIPaxDistributionAction": actions.SetCKIPaxDistributionAction.extract,
    "StoreAircraftDataAction": actions.StoreAircraftDataAction.extract,
    "StorePaxDataAction": actions.StorePaxDataAction.extract,
    "StorePaxDataGuiAction": actions.StorePaxDataAction.extract,
    "StoreRegistrationAndConfigurationAc": actions.StoreRegistrationAndConfigurationAc.extract,
    "TdmCreateLoadingInstructionAction": actions.TdmCreateLoadingInstructionAction.extract,
    "TransferCargoAction": actions.TransferCargoAction.extract,
    "TransferCheckinDataAction": actions.TransferCheckinDataAction.extract,
    "UpdateEstimatesAction": actions.UpdateEstimatesAction.extract,
    "UpdateFuelDataAction": actions.UpdateFuelDataAction.extract,
    "UpdateLoadTableAction": actions.UpdateLoadTableAction.extract,
    "UpdateTransitLoadTableAction": actions.UpdateTransitLoadTableAction.extract,
}

In [57]:
def extract_df(
    df: pd.DataFrame, progress_bar: bool = False, label: str | None = None
) -> pd.DataFrame:
    """Extract specific data based on predefined action extractors, optionally displaying a progress bar and labels.

    This function iterates over a dictionary of action names and their associated extractor functions, applying each
    extractor to the relevant entries in the DataFrame. The results are stored in new columns in the DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame from which data will be extracted.
            It must contain columns that match the keys in the action_extractors dictionary.
        progress_bar (bool, optional): If True, displays a progress bar during the data extraction process.
            Useful for visual feedback during long operations. Defaults to False.
        label (str | None, optional): An optional label that prefixes the print statements for better traceability during debugging.
            If None, only the action name is printed. Defaults to None.

    Returns:
        pd.DataFrame: The original DataFrame with additional columns containing the extracted data.
    """

    if progress_bar:
        tqdm.pandas()
    for action_name, extractor in action_extractors.items():
        if extractor is not None:
            if label:
                print(label, action_name)
            else:
                print(action_name)
            if progress_bar:
                df[f"data_{action_name}"] = df[df.action_name == action_name][
                    "entry_details"
                ].progress_apply(extractor)
            else:
                df[f"data_{action_name}"] = df[df.action_name == action_name][
                    "entry_details"
                ].apply(extractor)

    return df

In [58]:
def process_df(df, file_path, label, progress_bar=False):

    df_conv = extract_df(
        df,
        progress_bar=progress_bar,
        label=label,
    )
    df_conv.to_parquet(file_path, engine="pyarrow", compression="brotli")

In [59]:

# Create a ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [
        executor.submit(process_df, df_abcd, parquet_file_abcd_conv, "ABCD"),
        executor.submit(process_df, df_mnop, parquet_file_mnop_conv, "MNOP"),
        executor.submit(process_df, df_zyxw, parquet_file_zyxw_conv, "ZYXW"),
    ]

    # Optional: Wait for all futures to complete
    for future in futures:
        future.result()  # This will re-raise any exceptions that occurred during task execution

ABCD CalculateWeightAndTrimAction
MNOP CalculateWeightAndTrimAction
ZYXW CalculateWeightAndTrimAction
              id        creation_time airline_code  flight_number  \
46743   34496931  2024-05-07 09:16:01           AB           2384   
46744   34496930  2024-05-07 09:16:01           AB           2384   
46745   34496925  2024-05-07 09:16:01           AB           2562   
46746   34496924  2024-05-07 09:16:01           AB           2562   
46747   34496840  2024-05-07 09:15:01           AB           2108   
...          ...                  ...          ...            ...   
278728  33810074  2024-04-30 13:05:00           AB           2439   
278729  33810071  2024-04-30 13:05:00           AB           2439   
278730  33810072  2024-04-30 13:05:00           AB           2439   
278731  33809782  2024-04-30 13:03:00           AB           2439   
278732  33809781  2024-04-30 13:03:00           AB           2439   

        flight_date departure_airport     user_name  \
46743         

In [ ]:
# print("ABCD")
# df_abcd_conv = extract_df(df_abcd, progress_bar=True)
# df_abcd_conv.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")
# print("MNOP")
# df_mnop_conv = extract_df(df_mnop, progress_bar=True)
# df_mnop_conv.to_parquet(parquet_file_mnop_conv, engine="pyarrow", compression="brotli")
# print("ZYXW")
# df_zyxw_conv = extract_df(df_zyxw, progress_bar=True)
# df_zyxw_conv.to_parquet(parquet_file_zyxw_conv, engine="pyarrow", compression="brotli")

In [36]:
df_zyxw.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details', 'data_CalculateWeightAndTrimAction',
       'data_CheckinMsgProcessor', 'data_CreateLoadingInstructionAction',
       'data_CreateLoadSheetAction', 'data_CreateZFWMessageAction',
       'data_EstimateStorePaxDataAction', 'data_RampFinalAction',
       'data_SendFuelOrderAction', 'data_SendLoadingInstructionAction',
       'data_SendLoadsheetAction', 'data_SetActualBagWeightIndicatorAction',
       'data_SetCKIPaxDistributionAction', 'data_StoreAircraftDataAction',
       'data_StorePaxDataAction', 'data_StorePaxDataGuiAction',
       'data_StoreRegistrationAndConfigurationAc',
       'data_TdmCreateLoadingInstructionAction', 'data_TransferCargoAction',
       'data_TransferCheckinDataAction', 'data_UpdateEstimatesAction',
       'data_UpdateFuelDataAction', 'data_UpdateLoadTableAction',
       'data_UpdateTransitLoa

In [31]:
df_zyxw[df_zyxw.flight_date == 13].creation_time.apply(lambda x: str(x)[0:10]).unique()

array(['2024-05-06'], dtype=object)

In [35]:
print(df_abcd.creation_time.apply(lambda x: str(x)[0:10]).unique())
print(df_mnop.creation_time.apply(lambda x: str(x)[0:10]).unique())
print(df_zyxw.creation_time.apply(lambda x: str(x)[0:10]).unique())

['2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04' '2024-05-06'
 '2024-05-05' '2024-04-30' '2024-05-07']
['2024-04-30' '2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04'
 '2024-05-05' '2024-05-06' '2024-05-07']
['2024-04-30' '2024-05-01' '2024-05-02' '2024-05-03' '2024-05-04'
 '2024-05-05' '2024-05-06' '2024-05-07']


In [137]:
def extract(message: str):
    

    if (
        "Message type        :   LOADSHEET" in message
        or "Message type        :   LOADING_INSTRUCTION" in message
    ):
        """{
                'TOTAL TRAFFIC LOAD'
                'DRY OPERATING WEIGHT'
                'ZERO FUEL WEIGHT ACTUAL'
                'TAKE OFF FUEL'
                'TAKE OFF WEIGHT ACTUAL'
                'TRIP'
                'LANDING WEIGHT ACTUAL'
                },"""
        
        
        patterns = {}
        if "TRIP FUEL" in message:
            # This identifies zyxw messages because they are the only ones with TRIP followed by FUEL
            patterns = {
                "TOTAL TRAFFIC LOAD": r"TOTAL\s+TRAFFIC\s+LOAD\s+(\d+)",
                "DRY OPERATING WEIGHT": r"DRY\s+OPERATING\s+WEIGHT\s+(\d+)",
                "ZERO FUEL WEIGHT ACTUAL": r"ZERO\s+FUEL\s+WEIGHT\s+ACTUAL\s+(\d+)",
                "TAKE OFF FUEL": r"TAKE\s+OFF\s+FUEL\s+(\d+)",
                "TAKE OFF WEIGHT ACTUAL": r"TAKE\s+OFF\s+WEIGHT\s+ACTUAL\s+(\d+)",
                "TRIP FUEL": r"TRIP\s+FUEL\s+(\d+)",
                "LANDING WEIGHT ACTUAL": r"LANDING\s+WEIGHT\s+ACTUAL\s+(\d+)",
            }

        elif "ZERO FUEL WEIGHT ACTUAL" in message: # This identifies abcd messages
          
            patterns = {
                "TOTAL TRAFFIC LOAD": r"TOTAL TRAFFIC LOAD\s+(\d+)",
                "DRY OPERATING WEIGHT": r"DRY OPERATING WEIGHT\s+(\d+)",
                "ZERO FUEL WEIGHT ACTUAL": r"ZERO FUEL WEIGHT ACTUAL\s+(\d+)",
                "TAKE OFF FUEL": r"TAKE OFF FUEL\s+(\d+)",
                "TAKE OFF WEIGHT ACTUAL": r"TAKE OFF WEIGHT ACTUAL\s+(\d+)",
                "TRIP FUEL": r"TRIP\s+(\d+)",
                "LANDING WEIGHT ACTUAL": r"LANDING WEIGHT ACTUAL\s+(\d+)",
            }
        elif "ZFW" in message:
            # This identifies mnop messages because they are the only ones with abbreviations like ZFW
            patterns = {
            "DRY OPERATING WEIGHT": r"DOW\s+(\d+)", # DOW
            "TOTAL TRAFFIC LOAD": r"PAY\s+(\d+)", # PAY
            "ZERO FUEL WEIGHT ACTUAL": r"ZFW\s+(\d+)", # ZFW
            "TAKE OFF FUEL": r"TOF\s+(\d+)", # TOF
            "TAKE OFF WEIGHT ACTUAL": r"TOW\s+(\d+)", # TOW
            "TRIP FUEL": r"TIF\s+(\d+)", # TIF
            "LANDING WEIGHT ACTUAL": r"LAW\s+(\d+)" # LAW
            }
        
        extracted_weights = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, message)
            if match:
                extracted_weights[key] = int(match.group(1))
            else:
                extracted_weights[key] = None  # Assign None if the value is not found
        
        return json.dumps(extracted_weights)
    if "STATUS LOADSHEET" in message or "STATUS LOADING_INSTRUCTION" in message:
        return None
    if "lc2.common.dto.SingleAttributeDTO" in message:
        return None
    pass

In [14]:
# Example on how to effectively filter out different variations of the same message
df = df_abcd
df["entry_details"] = df["entry_details"].apply(remove_typos.remove_typos)

x = df[
    (df.action_name == "CreateLoadsheetAction")
    & (~df["entry_details"].isnull())
    & (
        ~df["entry_details"].str.contains("lc2.common.dto.SingleAttributeDTO", na=False)
    )  # abcd
    # & (
    #     ~df["entry_details"].str.contains("Message type        :   LOADSHEET", na=False)
    # )  # abcd
    # & (
    #     df["entry_details"].str.contains(
    #         "Message type        :   LOADING_INSTRUCTION", na=False
    #     )
    # )  # abcd
    & (~df["entry_details"].str.contains("STATUS LOADSHEET", na=False))  # abcd
    & (~df["entry_details"].str.contains("STATUS LOADING_INSTRUCTION", na=False))  # abcd
    # nMessage type        :   LOADING_INSTRUCTION
    # & (
    #     ~df["entry_details"].str.contains(
    #         "com.systemone.lc2.estimateshandling.dto.EstimateWeightsDTO", na=False
    #     )
    # )  # zyxw
    # & (
    #     ~df["entry_details"].str.contains("STATUS LOADING_INSTRUCTION", na=False)
    # )  # STATUS LOZYING_INSTRUCTION also included
    # & (~df["entry_details"].str.contains("STATUS LOZYING_INSTRUCTION", na=False))
    # & (
    #     ~df["entry_details"].str.contains("Email receivers", na=False)
    # )  # don't include this
    # & (
    #     ~df["entry_details"].str.contains("Telex receivers", na=False)
    # )  # don't include this
]["entry_details"]

In [15]:
print(CreateLoadsheetAction.extract(x.iloc[3]))

{"TOTAL TRAFFIC LOAD": 6779, "DRY OPERATING WEIGHT": 44831, "ZERO FUEL WEIGHT ACTUAL": 51610, "TAKE OFF FUEL": 8114, "TAKE OFF WEIGHT ACTUAL": 59724, "TRIP FUEL": 4524, "LANDING WEIGHT ACTUAL": 55200}


In [136]:
list(x)

['Telex receivers     :   ASRV1ZY\r\nMessage type        :   LOADSHEET\r\nSubject             :   ZY999/30APR GRU-REC; Loadsheet Edition 01\r\nAttachment file name:   ZY999_30APR_GRUREC_Loadsheet_Edition_01\r\nBody                :\r\nZYXW  AIRLINES    OFP:_____  ACCEPTED: _____\r\nL O A D S H E E T              CHECKED            APPROVED     EDNO\r\nALL WEIGHTS IN KILOGRAM                                         01\r\n\r\nFROM/TO FLIGHT      A/C REG     VERSION       CREW     DATE    TIME\r\nGRU REC ZY 999 /30 ZYXRD       Y174          3/4      30APR24 0757\r\n                        WEIGHT             DISTRIBUTION\r\nLOAD IN COMPARTMENTS      5420   1/2210 3/2000 4/1210 5/0\r\nPASSENGER / CABIN BAG    13260  170/0/0          TTL 170\r\n                                Y  170         \r\nTOTAL TRAFFIC LOAD       18680\r\nDRY OPERATING WEIGHT     45204\r\nZERO FUEL WEIGHT ACTUAL  63884  MAX  64300 L  ZYJ\r\nTAKE OFF FUEL             8809\r\nTAKE OFF WEIGHT  ACTUAL  72693  MAX  79000   

DOW 43369  DRY OPERATING WEIGHT
PAY 8805  TOTAL TRAFFIC LOAD
ZFW 52174  MAX   62500    ZERO FUEL WEIGHT ACTUAL
TOF 9674   TAKE OFF FUEL

TOW 61848  MAX   73500   TAKE OFF WEIGHT ACTUAL

TIF 4847   TAXI FUEL
LAW 57001  MAX   66000  L LANDING WEIGHT ACTUAL
UNLD 8999  

In [72]:
print(extract(x.unique()[0]))

{'TOTAL TRAFFIC LOAD': 12997, 'DRY OPERATING WEIGHT': 44966, 'ZERO FUEL WEIGHT ACTUAL': 57963, 'TAKE OFF FUEL': 5514, 'TAKE OFF WEIGHT ACTUAL': 63477, 'TRIP': 2212, 'LANDING WEIGHT ACTUAL': 61265, 'TAXI FUEL': 286, 'PANTRY': 414}


In [56]:
y = pd.DataFrame()

In [57]:
y["entry_details"] = pd.DataFrame(x)

In [58]:
y["data"] = pd.DataFrame(x.apply(extract))

In [59]:
y

,entry_details,data
423236,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 15200, 'DRY OPERATING W..."
423251,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14975, 'DRY OPERATING W..."
423272,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14701, 'DRY OPERATING W..."
423278,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 6779, 'DRY OPERATING WE..."
423303,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 18706, 'DRY OPERATING W..."
...,...,...
448385,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 14942, 'DRY OPERATING W..."
448396,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 15032, 'DRY OPERATING W..."
448411,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 8028, 'DRY OPERATING WE..."
448425,\r\nEmail receivers :\r\nMessage type ...,"{'TOTAL TRAFFIC LOAD': 18430, 'DRY OPERATING W..."


In [28]:
print(extract(x[0]))

{'TOTAL TRAFFIC LOAD': 15200, 'DRY OPERATING WEIGHT': 45523, 'ZERO FUEL WEIGHT ACTUAL': 60723, 'TAKE OFF FUEL': 12502, 'TAKE OFF WEIGHT ACTUAL': 73225, 'TRIP': 4808, 'LANDING WEIGHT ACTUAL': 68417, 'TAXI FUEL': 198, 'PANTRY': 414}
